# Generazione dei dataset nelle differenti lingue

In questo file vengono presi i 4 dataset nelle 4 differenti lingue e vengono estratte le triple comuni, in quanto il dataset tedesco e russo non sono allineati a quello inglese e italiano (contenogno meno triple, addirittura il tedesco non ha il test set)

Successivamente, viene estratto un set di 1779 entry (in linea con la cardinalità del test set della challenge 2020), che andrà a formare il test set su cui valutare l'esperimento

In [65]:
import os
import sentencepiece as spm
import pandas as pd
import numpy as np
from tqdm import tqdm

## Recupero dei record dei dataset

In [66]:
dataset_types = ["ge", "en", "it", "ru"]

# Salva i dataset nella variabile datasets
datasets = {}
for dataset_type in dataset_types:
    datasets[dataset_type] = pd.read_csv("datasets\\" + dataset_type + "_dataset.csv")
    print(f'{dataset_type} dataset caricato. Lunghezza: {len(datasets[dataset_type])}')

ge dataset caricato. Lunghezza: 7812
en dataset caricato. Lunghezza: 16657
it dataset caricato. Lunghezza: 16657
ru dataset caricato. Lunghezza: 7465


## Normalizzazione delle triple

Ciò viene fatto per trovare più triple comuni

In [67]:
# Funzione per normalizzare le stringhe
def normalize_string(s):
    if isinstance(s, str):
        # replace &quot; with "
        s = s.replace("&quot;", "\"")
        return s.lower().replace("’", "'").replace("‘", "'").strip()
    return s

# Normalizza le colonne "data_unit" in tutti i dataset
for dataset_type in dataset_types:
    datasets[dataset_type]["data_unit_pe"] = datasets[dataset_type]["data_unit"].apply(normalize_string)

In [68]:
datasets["ge"].head()

,eid,size,dataset_type,category,shape_type,data_unit,sentence,data_unit_pe
0,Id1,1,train,Airport,NaN,"Aarhus_Airport cityServed ""Aarhus, Denmark""","['Das Aarhus ist der Flughafen von Aarhus, Dän...","aarhus_airport cityserved ""aarhus, denmark"""
1,Id2,1,train,Airport,NaN,Aarhus_Airport cityServed Aarhus,['Der Flughafen Aarhus dient der Stadt Aarhus. '],aarhus_airport cityserved aarhus
2,Id3,1,train,Airport,NaN,Aarhus_Airport elevationAboveTheSeaLevel_(in_m...,['Der Flughafen Aarhus liegt 25 Meter über dem...,aarhus_airport elevationabovethesealevel_(in_m...
3,Id4,1,train,Airport,NaN,Aarhus_Airport location Tirstrup,['Der Flughafen Aarhus befindet sich in Tirstr...,aarhus_airport location tirstrup
4,Id5,1,train,Airport,NaN,"Aarhus_Airport operatingOrganisation ""Aarhus L...",['Der Flughafen Aarhus wird von Aarhus Lufthav...,"aarhus_airport operatingorganisation ""aarhus l..."


## Recupero delle triple comuni

In [69]:
main = "ge"
others = ["it", "en", "ru"]

# Crea un set iniziale con i data_unit_pe del dataset principale
common_data_units = set(datasets[main]["data_unit_pe"])

# Interseca con i data_unit_pe di ciascun altro dataset
for other in others:
    common_data_units.intersection_update(set(datasets[other]["data_unit_pe"]))

# Numero di data_unit comuni
len_common = len(common_data_units)
print(f"Numero di data units comuni a tutti i dataset: {len_common}")

Numero di data units comuni a tutti i dataset: 3828


In [70]:
common_data_units = list(common_data_units)
common_data_units[:5]

['united_kingdom currency pound_sterling 20_fenchurch_street location united_kingdom',
 'john_van_den_brom club afc_ajax',
 'ayam_penyet country java java ethnicgroup baduy singapore language english_language ayam_penyet region singapore',
 'michele_marcolini club a.c._lumezzane',
 'appleton_international_airport cityserved appleton,_wisconsin greenville,_wisconsin country united_states greenville,_wisconsin ispartof grand_chute,_wisconsin greenville,_wisconsin ispartof clayton,_winnebago_county,_wisconsin appleton_international_airport location greenville,_wisconsin']

In [71]:
print("WebNLG-IT Length: ", len(datasets["it"]))
print("WebNLG-GE Length: ", len(datasets["ge"]))
print("WebNLG-EN Length: ", len(datasets["en"]))
print("WebNLG-RU Length: ", len(datasets["ru"]))
print("Common data units: ", len(common_data_units))

WebNLG-IT Length:  16657
WebNLG-GE Length:  7812
WebNLG-EN Length:  16657
WebNLG-RU Length:  7465
Common data units:  3828


## Estrazione Test set e creazione train e dev con le rimanenti triple per ogni lingua

In [72]:
# per ogni dataset, mantieni solo i record con le triple in common_data_units
new_datasets = {}
for dataset_type in dataset_types:
    new_datasets[dataset_type] = datasets[dataset_type][datasets[dataset_type]["data_unit_pe"].isin(common_data_units)]
    print(f'{dataset_type} dataset ridotto. Lunghezza: {len(new_datasets[dataset_type])}')

print()

# per ogni dataset, mantieni solo la prima occorrenza di ogni data_unit_pe
for dataset_type in dataset_types:
    new_datasets[dataset_type] = new_datasets[dataset_type].drop_duplicates(subset="data_unit_pe")
    print(f'{dataset_type} dataset ridotto. Lunghezza: {len(new_datasets[dataset_type])}')



ge dataset ridotto. Lunghezza: 3871
en dataset ridotto. Lunghezza: 3904
it dataset ridotto. Lunghezza: 3904
ru dataset ridotto. Lunghezza: 3845

ge dataset ridotto. Lunghezza: 3828
en dataset ridotto. Lunghezza: 3828
it dataset ridotto. Lunghezza: 3828
ru dataset ridotto. Lunghezza: 3828


In [73]:
import ast

# Funzione per sincronizzare le verbalizzazioni
def synchronize_verbalizations(new_datasets, common_data_units):
    # Itera su ogni data_unit comune
    for data_unit in tqdm(common_data_units, desc="Sincronizzazione delle verbalizzazioni"):
        # Ottieni le liste di verbalizzazioni per ogni dataset
        verbalizations = {
            "ge": [],
            "en": [],
            "it": [],
            "ru": []
        }
        # get for each new_datasets the row with the data_unit_pe == data_unit
        for dataset_type in dataset_types:
            sentences = new_datasets[dataset_type][new_datasets[dataset_type]["data_unit_pe"] == data_unit]["sentence"]
            if not sentences.empty:
                verbalizations[dataset_type] = ast.literal_eval(sentences.values[0])
                
        min_len = min([len(verbalizations[dataset_type]) for dataset_type in dataset_types])
        # controlla se le liste di verbalizzazioni sono tutte della stessa lunghezza
        #if len(set([len(verbalizations[dataset_type]) for dataset_type in dataset_types])) != 1:
        #    print(f"Le liste di verbalizzazioni per la data unit {data_unit} non sono della stessa lunghezza.")
        #    print("Lunghezze:")
        #    for dataset_type in dataset_types:
        #        print(f"{dataset_type}: {len(verbalizations[dataset_type])}")
        #    print()

        # Sincronizza le verbalizzazioni
        for dataset_type in dataset_types:
            # mantiene solo le prime min_len verbalizzazioni
            verbalizations[dataset_type] = verbalizations[dataset_type][:min_len]

        # Aggiorna i dataset
        for dataset_type in dataset_types:
            # Converti la lista troncata in stringa prima di assegnarla
            new_datasets[dataset_type].loc[
                new_datasets[dataset_type]["data_unit_pe"] == data_unit, "sentence"
            ] = str(verbalizations[dataset_type])

    return new_datasets

# Sincronizza le verbalizzazioni
new_datasets = synchronize_verbalizations(new_datasets, common_data_units)

# Verifica dei risultati
for dataset_type in dataset_types:
    print(f"Dataset {dataset_type} sincronizzato. Lunghezza: {len(new_datasets[dataset_type])}")




Sincronizzazione delle verbalizzazioni: 100%|██████████| 3828/3828 [00:30<00:00, 123.98it/s]

Dataset ge sincronizzato. Lunghezza: 3828
Dataset en sincronizzato. Lunghezza: 3828
Dataset it sincronizzato. Lunghezza: 3828
Dataset ru sincronizzato. Lunghezza: 3828


In [74]:
# print conta le sentences per dataset
for dataset_type in dataset_types:
    sentences = new_datasets[dataset_type]["sentence"]

    i = 0
    for sentence in sentences:
        sentence = ast.literal_eval(sentence)
        i += len(sentence)

    print(f"{dataset_type}: {i}")


ge: 9982
en: 9982
it: 9982
ru: 9982


In [75]:
#order by data_unit_pe
for dataset_type in dataset_types:
    new_datasets[dataset_type] = new_datasets[dataset_type].sort_values(by="data_unit_pe")

# aggiorna gli indici di tutti i dataset
for dataset_type in dataset_types:
    new_datasets[dataset_type].reset_index(drop=True, inplace=True)

In [78]:
# per ogni dataset crea train, dev e test set
train_ratio = 0.8
dev_ratio = 0.1
test_ratio = 0.1

train_datasets = {}
dev_datasets = {}
test_datasets = {}

for dataset_type in dataset_types:
    train_datasets[dataset_type] = new_datasets[dataset_type].sample(frac=train_ratio, random_state=42)
    dev_test = new_datasets[dataset_type].drop(train_datasets[dataset_type].index)
    dev_datasets[dataset_type] = dev_test.sample(frac=dev_ratio/(dev_ratio + test_ratio), random_state=42)
    test_datasets[dataset_type] = dev_test.drop(dev_datasets[dataset_type].index)

    print(f'{dataset_type} dataset:')
    print(f'Train: {len(train_datasets[dataset_type])}, Dev: {len(dev_datasets[dataset_type])}, Test: {len(test_datasets[dataset_type])}')

print()

# train e dev hanno un array di sentence, per ognuna di esse genera un record con data_unit_pe e sentence
train_data = {
    "ge": [],
    "en": [],
    "it": [],
    "ru": []
}
dev_data = {
    "ge": [],
    "en": [],
    "it": [],
    "ru": []
}

import ast

for dataset_type in dataset_types:
    for i, row in train_datasets[dataset_type].iterrows():
        array_sentences = ast.literal_eval(row["sentence"])

        for sentence in array_sentences:
            train_data[dataset_type].append({
                "eid": row["eid"],
                "size": row["size"],
                "dataset_type": dataset_type,
                "category": row["category"],
                "shape_type": row["shape_type"],
                "data_unit": row["data_unit"],
                "sentence": sentence,
            })

    for i, row in dev_datasets[dataset_type].iterrows():
        array_sentences = ast.literal_eval(row["sentence"])

        for sentence in array_sentences:
            dev_data[dataset_type].append({
                "eid": row["eid"],
                "size": row["size"],
                "dataset_type": dataset_type,
                "category": row["category"],
                "shape_type": row["shape_type"],
                "data_unit": row["data_unit"],
                "sentence": sentence,
            })

for dataset_type in dataset_types:
    train_data[dataset_type] = pd.DataFrame(train_data[dataset_type])
    dev_data[dataset_type] = pd.DataFrame(dev_data[dataset_type])

    # mischia
    train_data[dataset_type] = train_data[dataset_type].sample(frac=1, random_state=42)
    dev_data[dataset_type] = dev_data[dataset_type].sample(frac=1, random_state=42)
    test_datasets[dataset_type] = test_datasets[dataset_type].sample(frac=1, random_state=42)

    j = 0
    for i, row in test_datasets[dataset_type].iterrows():
        array_sentences = ast.literal_eval(row["sentence"])
        j += len(array_sentences)

    print(f'{dataset_type} dataset:')
    print(f'Train: {len(train_data[dataset_type])}, Dev: {len(dev_data[dataset_type])}, Test: {j}')

    

ge dataset:
Train: 3062, Dev: 383, Test: 383
en dataset:
Train: 3062, Dev: 383, Test: 383
it dataset:
Train: 3062, Dev: 383, Test: 383
ru dataset:
Train: 3062, Dev: 383, Test: 383

ge dataset:
Train: 7998, Dev: 991, Test: 993
en dataset:
Train: 7998, Dev: 991, Test: 993
it dataset:
Train: 7998, Dev: 991, Test: 993
ru dataset:
Train: 7998, Dev: 991, Test: 993


In [79]:
# Salva i dataset in formato csv
for dataset_type in dataset_types:
    train_data[dataset_type].to_csv(f'datasets_esperimento\\{dataset_type}\\train.csv', index=False)
    dev_data[dataset_type].to_csv(f'datasets_esperimento\\{dataset_type}\\dev.csv', index=False)
    test_datasets[dataset_type].to_csv(f'datasets_esperimento\\{dataset_type}\\test.csv', index=False)